<a href="https://colab.research.google.com/github/blackhawkee/llm-play/blob/main/stable_diffusion_xl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U autotrain-advanced
!autotrain setup --update-torch

In [ ]:
import requests
import os
images = [  "https://www.pngall.com/wp-content/uploads/4/Labrador-Retriever-Puppy-PNG-Free-Image.png",
            "https://www.pngall.com/wp-content/uploads/4/Labrador-Retriever-Puppy-PNG-High-Quality-Image.png",
            "https://www.pngall.com/wp-content/uploads/4/Labrador-Retriever-Puppy-PNG-Image-File.png",
            "https://www.pngall.com/wp-content/uploads/4/Labrador-Retriever-Puppy-Transparent.png",
            "https://www.pngall.com/wp-content/uploads/4/Labrador-Retriever-PNG-Download-Image.png",
            "https://www.pngall.com/wp-content/uploads/4/Labrador-Retriever-PNG-Transparent-HD-Photo.png",
            "https://www.pngall.com/wp-content/uploads/4/Labrador-Retriever-PNG.png",
            "https://www.pngall.com/wp-content/uploads/4/Labrador-Retriever-Puppy-PNG-Download-Image.png"
            ]

images_directory = "/content/images"
if not os.path.exists(images_directory):
    os.makedirs(images_directory)

for idx, url in enumerate(images):
  response = requests.get(url)
  with open(f"{images_directory}/image_{str(idx)}.jpg", "wb") as f:
      f.write(response.content)

In [ ]:
!autotrain dreambooth \
--model stabilityai/stable-diffusion-xl-base-1.0 \
--output model_output/ \
--image-path /content/images/ \
--prompt "photo of sks dog" \
--resolution 512 \
--batch-size 1 \
--num-steps 150 \
--fp16 \
--gradient-accumulation 4 \
--lr 1e-4 \
--use-8bit-adam

In [ ]:
from diffusers import DiffusionPipeline, StableDiffusionXLImg2ImgPipeline
import torch

model = "stabilityai/stable-diffusion-xl-base-1.0"
pipe = DiffusionPipeline.from_pretrained(
    model,
    torch_dtype = torch.float16
)
pipe.to("cuda")
pipe.load_lora_weights(
    "model_output/", weight_name = "pytorch_lora_weights.safetensors"
)

# refiner = StableDiffusionXLImg2ImgPipeline.from_pretrained(
#     "stabilityai/stable-diffusion-xl-refiner-1.0",
#     torch_dtype = torch.float16
# )
# refiner.to("cuda")

prompt = "portrait of sks dog, cartoon, pixar, 3d, headshots, fantasy, 4k, uhd"
for seed in range(30):
  generator = torch.Generator("cuda").manual_seed(seed)
  image = pipe(
      prompt=prompt, generator=generator, num_inference_steps=25
  )
  image = image.images[0]
  # image = refiner(prompt=prompt, generator=generator, image=image)
  image=image.images[0]
  image.save(f"images/{seed}.png")